In [1]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()

X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

In [2]:
from scipy.stats import reciprocal

param_distribs = {
"model__n_hidden": range(4),
"model__n_neurons": range(1, 101),
"model__learning_rate": reciprocal(3e-4, 3e-2).rvs(1000).tolist(),
"model__optimizer": ['adam', 'sgd', 'nesterov']
}

In [3]:
import tensorflow as tf
from tensorflow import keras

def build_model(n_hidden, n_neurons, optimizer, learning_rate):
    model = tf.keras.models.Sequential()
    model.add(keras.layers.InputLayer(X_train.shape[1]))
    for _ in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons))
    model.add(keras.layers.Dense(1))
    if optimizer == 'sgd':
        my_optimizer = keras.optimizers.SGD(learning_rate=learning_rate, clipvalue=0.9)
    elif optimizer == 'nesterov':
        my_optimizer = keras.optimizers.SGD(learning_rate=learning_rate, nesterov=True, clipvalue=0.9)
    elif optimizer == 'adam':
        my_optimizer = keras.optimizers.Adam(learning_rate=learning_rate, clipvalue=0.9)
    else:
        my_optimizer = keras.optimizers.get(optimizer, learning_rate=learning_rate, clipvalue=0.9)
    model.compile(loss='mse', optimizer=my_optimizer)
    return model

In [4]:
import scikeras
from scikeras.wrappers import KerasRegressor

es = tf.keras.callbacks.EarlyStopping(patience=10, min_delta=1.0, verbose=1)
keras_reg = KerasRegressor(build_model, callbacks=[es])

In [5]:
from sklearn.model_selection import RandomizedSearchCV

rnd_search_cv = RandomizedSearchCV(keras_reg, param_distribs, n_iter=10, cv=3, verbose=2)
rnd_search_cv.fit(X_train, y_train, epochs=100, validation_data=(X_valid, y_valid), verbose=0)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


2023-06-13 15:34:39.579882: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Epoch 22: early stopping
121/121 [==============================] - 0s 285us/step
[CV] END model__learning_rate=0.0005524317430207671, model__n_hidden=2, model__n_neurons=51, model__optimizer=adam; total time=   3.2s
Epoch 11: early stopping
121/121 [==============================] - 0s 281us/step
[CV] END model__learning_rate=0.0005524317430207671, model__n_hidden=2, model__n_neurons=51, model__optimizer=adam; total time=   1.7s
Epoch 16: early stopping
121/121 [==============================] - 0s 335us/step
[CV] END model__learning_rate=0.0005524317430207671, model__n_hidden=2, model__n_neurons=51, model__optimizer=adam; total time=   2.6s
Epoch 21: early stopping
121/121 [==============================] - 0s 298us/step
[CV] END model__learning_rate=0.006127962092685077, model__n_hidden=3, model__n_neurons=71, model__optimizer=adam; total time=   3.3s
Epoch 13: early stopping
121/121 [==============================] - 0s 302us/step
[CV] END model__learning_rate=0.006127962092685077,

RandomizedSearchCV(cv=3,
                   estimator=KerasRegressor(callbacks=[<keras.callbacks.EarlyStopping object at 0x169d77970>], model=<function build_model at 0x11e1b4d30>),
                   param_distributions={'model__learning_rate': [0.0061774637391925685,
                                                                 0.00039029074011143016,
                                                                 0.02062463835678116,
                                                                 0.006550753333505036,
                                                                 0.0008443282187165011,
                                                                 0.00040550618158762724,
                                                                 0.006590622712725471...
                                                                 0.0010367426599185935,
                                                                 0.006710435190817113,
                                                                 0.000785294977408029,
                                                                 0.012377781995481014,
                                                                 0.0006930168588329453,
                                                                 0.010015299160888681,
                                                                 0.020856032037687846,
                                                                 0.004377480313174592,
                                                                 0.008992856950917501,
                                                                 0.006127962092685077,
                                                                 0.009408004789402678, ...],
                                        'model__n_hidden': range(0, 4),
                                        'model__n_neurons': range(1, 101),
                                        'model__optimizer': ['adam', 'sgd',
                                                             'nesterov']},
                   verbose=2)

In [6]:
import pickle

with open('rnd_search_params.pkl', 'wb') as params_pickle:
    pickle.dump(rnd_search_cv.best_params_, params_pickle)

with open('rnd_search_scikeras.pkl', 'wb') as scikeras_pickle:
    pickle.dump(rnd_search_cv, scikeras_pickle)

INFO:tensorflow:Assets written to: ram://71f987a0e44f47efb56e50ec1903eeac/assets


INFO:tensorflow:Assets written to: ram://71f987a0e44f47efb56e50ec1903eeac/assets


In [7]:
import keras_tuner as kt

def build_model_kt(hp):
    n_hidden = hp.Int("n_hidden", min_value=0, max_value=3, default=2)
    n_neurons = hp.Int("n_neurons", min_value=1, max_value=100)
    learning_rate = hp.Float("learning_rate", min_value=3e-4, max_value=3e-2, sampling="log")
    optimizer = hp.Choice("optimizer", values=["sgd", "adam", "nesterov"])

    if optimizer == 'sgd':
        my_optimizer = keras.optimizers.SGD(learning_rate=learning_rate, clipvalue=0.9)
    elif optimizer == 'nesterov':
        my_optimizer = keras.optimizers.SGD(learning_rate=learning_rate, nesterov=True, clipvalue=0.9)
    elif optimizer == 'adam':
        my_optimizer = keras.optimizers.Adam(learning_rate=learning_rate, clipvalue=0.9)
    else:
        my_optimizer = keras.optimizers.get(optimizer, learning_rate=learning_rate, clipvalue=0.9)

    model = tf.keras.models.Sequential()
    model.add(keras.layers.InputLayer(X_train.shape[1]))
    for _ in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons))
    model.add(keras.layers.Dense(1))
    model.compile(loss='mse', optimizer=my_optimizer)
    return model

In [8]:
random_search_tuner = kt.RandomSearch(build_model_kt, objective="val_loss", max_trials=10, overwrite=True,
directory="my_california_housing", project_name="my_rnd_search", seed=42)

In [9]:
import os

root_logdir = os.path.join(random_search_tuner.project_dir, 'tensorboard')
tb = tf.keras.callbacks.TensorBoard(root_logdir)

In [10]:
%load_ext tensorboard
%tensorboard --logdir {root_logdir}
random_search_tuner.search(X_train, y_train, epochs=100, validation_data=(X_valid, y_valid), callbacks=[tb, es])

Trial 10 Complete [00h 00m 04s]
val_loss: 7.694312572479248

Best val_loss So Far: 0.7939809560775757
Total elapsed time: 00h 00m 36s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


In [11]:

params = random_search_tuner.get_best_hyperparameters()[0]

best_hyperparameters = {
    'n_hidden': params.get('n_hidden'),
    'n_neurons': params.get('n_neurons'),
    'learning_rate': params.get('learning_rate'),
    'optimizer': params.get('optimizer')
}

best_hyperparameters

{'n_hidden': 1,
 'n_neurons': 25,
 'learning_rate': 0.027153822293473857,
 'optimizer': 'adam'}

In [12]:
with open('kt_search_params.pkl', 'wb') as params_keras:
    pickle.dump(best_hyperparameters, params_keras)

In [13]:
best_model = random_search_tuner.get_best_models()[0]
best_model.save('kt_best_model.h5')